In [2]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

# Path to your service account JSON key
SERVICE_ACCOUNT_FILE = "service_account.json"

# Scopes: read-only access
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# Authenticate
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# Spreadsheet ID (from your URL)
SHEET_ID = "1zX5r8SaqnNqOsQRxqSfB2tuRqoONQMhrtpHQUfVLwxs"
sh = client.open_by_key(SHEET_ID)

# --- Load Masterlist ---
ws_master = sh.worksheet("Masterlist")  # exact tab name
rows_master = ws_master.get_all_values()
df_master = pd.DataFrame(rows_master[1:], columns=rows_master[0])

# --- Load VIP ---
ws_vip = sh.worksheet("VIP")  # exact tab name
rows_vip = ws_vip.get_all_values()
df_vip = pd.DataFrame(rows_vip[1:], columns=rows_vip[0])

# Print first few rows from each
# print("Masterlist:")
# print(df_master.head())

# print("\nVIP:")
# print(df_vip.head())


/Users/jason/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
df_master.columns

Index(['Name', 'Local Hospital Network (LHN)', 'Granularity', 'Status',
       'Horizon\n(DO NOT EDIT)', 'Org_ID', 'Record ID', 'Provider Number',
       'State', 'Sector', 'Street Address', 'Suburb', 'Postcode', 'Grouping',
       'Declared category', 'Number of beds', 'Exist in 2025',
       'Number of bed (Actual)', 'OLD_Public_Org_ID',
       'Number of clincial staff', 'AIHW Ref#', 'Notes', 'Data Source'],
      dtype='object')

In [4]:
import pandas as pd
import numpy as np

def transform_hierarchy_preserve(df_in: pd.DataFrame, drop_empty_parents: bool = True) -> pd.DataFrame:
    """
    Build State → LHN → Facility hierarchy.
    Preserves 'Number of clincial staff' / 'Number of clinical staff'
    values exactly as given in the source (no aggregation).
    """
    df = df_in.copy()

    # Normalize key columns
    for col in ['Granularity','State','Name','Local Hospital Network (LHN)']:
        if col not in df.columns:
            df[col] = ""
        df[col] = df[col].astype(str).str.strip()

    g = df['Granularity'].str.casefold()

    # ---- Column name detection ----
    bed_col   = 'Number of bed (Actual)' if 'Number of bed (Actual)' in df.columns else '# of bed (Actual)'
    staff_col = 'Number of clincial staff' if 'Number of clincial staff' in df.columns else 'Number of clinical staff'
    sector_col = 'Sector' if 'Sector' in df.columns else None
    status_col = 'Status' if 'Status' in df.columns else None
    notes_col  = 'Notes' if 'Notes' in df.columns else None
    extra_cols = [c for c in [sector_col, bed_col, staff_col, status_col, notes_col] if c]

    # --- Order maps (preserve first appearance) ---
    state_order = df.loc[g.eq('state'),'State'].drop_duplicates().tolist()
    state_pos = {s:i for i,s in enumerate(state_order)}

    lhn_pos, seen_lhn = {}, {}
    for _, r in df[g.eq('lhn')][['State','Name']].iterrows():
        s, l = r['State'], r['Name']
        if (s,l) not in lhn_pos:
            seen_lhn.setdefault(s,0)
            lhn_pos[(s,l)] = seen_lhn[s]
            seen_lhn[s]+=1

    fac_pos, seen_fac = {}, {}
    for _, r in df[g.eq('facility')][['State','Local Hospital Network (LHN)','Name']].iterrows():
        key = (r['State'], r['Local Hospital Network (LHN)'])
        name = r['Name']
        if (key[0], key[1], name) not in fac_pos:
            seen_fac.setdefault(key,0)
            fac_pos[(key[0],key[1],name)] = seen_fac[key]
            seen_fac[key]+=1

    # --- Build hierarchy rows ---
    states = (
        df[g.eq('state')]
        .drop_duplicates(subset=['State'])
        .rename(columns={'Name':'State_Name'})
        .assign(**{
            'Local Hospital Network (LHN)':'',
            'Hospital Name':'',
            'Granularity':'State'
        })[['State','Local Hospital Network (LHN)','Hospital Name','Granularity']+extra_cols]
    )

    lhns = (
        df[g.eq('lhn')][['State','Name']+extra_cols].drop_duplicates()
        .rename(columns={'Name':'Local Hospital Network (LHN)'})
        .assign(**{
            'Hospital Name':'',
            'Granularity':'LHN'
        })[['State','Local Hospital Network (LHN)','Hospital Name','Granularity']+extra_cols]
    )

    facilities = (
        df[g.eq('facility')]
        .rename(columns={'Name':'Hospital Name'})
        [['State','Local Hospital Network (LHN)','Hospital Name','Granularity']+extra_cols]
        .assign(Granularity='Facility')
    )

    # Drop empty parents if requested
    if drop_empty_parents:
        valid_lhns = set(map(tuple, facilities[['State','Local Hospital Network (LHN)']].drop_duplicates().to_numpy()))
        lhns = lhns[lhns.apply(lambda r: (r['State'], r['Local Hospital Network (LHN)']) in valid_lhns, axis=1)]
        valid_states = set(lhns['State']).union(set(facilities['State']))
        states = states[states['State'].isin(valid_states)]

    out = pd.concat([states, lhns, facilities], ignore_index=True)

    # Sort
    def sort_key(r):
        s,l,h,g = r['State'], r['Local Hospital Network (LHN)'], r['Hospital Name'], r['Granularity'].casefold()
        s_idx = state_pos.get(s,10**9)
        if g=='state': return (s_idx,-1,-1)
        elif g=='lhn': return (s_idx,lhn_pos.get((s,l),10**9),-1)
        else: return (s_idx,lhn_pos.get((s,l),10**9),fac_pos.get((s,l,h),10**9))

    out['_k'] = out.apply(sort_key, axis=1)
    out = out.sort_values('_k').drop(columns=['_k']).reset_index(drop=True)
    return out


def format_hierarchy_for_display(df_hier: pd.DataFrame) -> pd.DataFrame:
    """Blank repeated parents for nicer view, keep extra columns."""
    df = df_hier.copy()

    df['State (display)'] = df['State']
    df['LHN (display)']   = df['Local Hospital Network (LHN)']
    df['Hospital (display)'] = df['Hospital Name']

    df.loc[df['Granularity']!='State','State (display)']=''
    df.loc[df['Granularity']=='Facility','LHN (display)']=''
    df.loc[df['Granularity']!='Facility','Hospital (display)']=''

    keep = ['State (display)','LHN (display)','Hospital (display)','Granularity']
    extra_cols = [c for c in df.columns if c not in keep+['State','Local Hospital Network (LHN)','Hospital Name']]
    return df[keep+extra_cols].rename(columns={
        'State (display)':'State',
        'LHN (display)':'Local Hospital Network (LHN)',
        'Hospital (display)':'Hospital Name'
    })


In [5]:
# Filter to PUBLIC sector only
df_public = df_master[df_master['Sector'].astype(str).str.strip().str.upper()=='PUBLIC'].copy()

# Build hierarchy, preserving staff values
df_hier = transform_hierarchy_preserve(df_public, drop_empty_parents=True)

# Format for display
df_view = format_hierarchy_for_display(df_hier)

# Show first rows
display(df_view.head(30))


,State,Local Hospital Network (LHN),Hospital Name,Granularity,Sector,Number of bed (Actual),Number of clincial staff,Status,Notes
0,ACT,,,State,PUBLIC,,3606,,
1,,Australian Capital Territory,,LHN,PUBLIC,,,Untouched,
2,,,QEII FAMILY CENTRE,Facility,PUBLIC,<50,,,
3,,,THE CANBERRA HOSPITAL,Facility,PUBLIC,>500,,,
4,,,CALVARY PUBLIC HOSPITAL,Facility,PUBLIC,200-500,,Discovery,Strong IMedX relationship
5,,,UNIVERSITY OF CANBERRA HOSPITAL,Facility,PUBLIC,,,,
6,NSW,,,State,PUBLIC,,"115,503",,
7,,Central Coast Local Health District,,LHN,PUBLIC,,5999,Untouched,
8,,,GOSFORD HOSPITAL,Facility,PUBLIC,200-500,,,
9,,,WOY WOY HOSPITAL,Facility,PUBLIC,50-99,,,


In [6]:
df_view.to_csv("Masterlist_HIER.csv",index=False)

In [94]:
df_vip.columns


Index(['Org', 'State', 'Name', 'Company Record ID', 'Org_ID', 'Title',
       'Organisation', 'Email', 'Owner', 'Note'],
      dtype='object')